<a href="https://colab.research.google.com/github/Mahdi-Saadati/Machine-Learning-2/blob/main/3.c.%20Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div dir="rtl" align="right">
تشخیص ارقام دستنویس از روی دیتاست بارگزاری شده بر روی کامپیوتر

In [ ]:
import numpy as np                     # کتابخانه عددی برای آرایه‌ها و محاسبات
import matplotlib.pyplot as plt        # برای رسم نمودارها
import os                              # کار با مسیرها و فایل‌ها
from imutils import paths              # برای گرفتن مسیر تمام تصاویر داخل یک پوشه
import cv2                             # کتابخانه OpenCV برای پردازش تصویر
from keras import backend as K         # بک‌اند Keras برای تنظیمات داخلی
from sklearn.model_selection import train_test_split  # تقسیم داده‌ها به آموزش/تست
from keras.utils import to_categorical # برای تبدیل برچسب‌ها به دسته‌ای (one-hot)
from keras.models import Sequential    # ساخت مدل ترتیبی شبکه عصبی
from keras.layers import Conv2D        # لایه کانولوشن
from keras.layers import MaxPooling2D  # لایه Pooling
from keras.layers import Flatten       # تخت کردن (تبدیل 2D به 1D)
from keras.layers import Dense         # لایه کاملاً متصل
from keras.layers import Dropout       # جلوگیری از overfitting
from keras.optimizers import Adam      # بهینه‌ساز آدم

# تابع برای بارگذاری دیتاست تصاویر
def Loading_Image_dataset(Link):
    Img_dataset = []                          # لیست تصاویر
    Img_labels = []                           # لیست برچسب‌ها
    imagePaths = list(paths.list_images(Link)) # گرفتن تمام مسیرهای تصاویر
    for imagePath in imagePaths:               # پیمایش تک‌تک تصاویر
       image = cv2.imread(imagePath, cv2.IMREAD_UNCHANGED)  # خواندن تصویر
       label = int(imagePath.split(os.path.sep)[-2])        # پوشه قبل از تصویر = برچسب
       image = image.astype('float32')        # تبدیل نوع داده به float32
       Img_dataset.append(image)              # افزودن تصویر به لیست
       Img_labels.append(label)               # افزودن برچسب به لیست
    Img_dataset_array = np.asarray(Img_dataset) # لیست → آرایه numpy
    Img_labels_array = np.asarray(Img_labels)   # لیست → آرایه numpy
    return Img_dataset_array, Img_labels_array  # خروجی: داده‌ها و برچسب‌ها


Link = "DigitDataset"                  # مسیر دیتاست
X, L = Loading_Image_dataset(Link)     # بارگذاری تصاویر و برچسب‌ها

# بررسی نوع کانال تصویر (channels_first یا channels_last)
if K.image_data_format() == 'channels_first':
    X = X.reshape(X.shape[0], 1, X.shape[1], X.shape[2])     # (نمونه‌ها، کانال، طول، عرض)
    input_shape = (1, X.shape[1], X.shape[2])
else:
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)     # (نمونه‌ها، طول، عرض، کانال)
    input_shape = (X.shape[1], X.shape[2], 1)

# تقسیم داده به آموزش و تست (۷۰٪ آموزش، ۳۰٪ تست)
X_train, X_test, L_train, L_test = train_test_split(X, L, test_size=0.3)

# تبدیل برچسب‌ها به one-hot (مثلاً 3 → [0,0,0,1,0,...])
L_train = to_categorical(L_train, num_classes=10)
L_test = to_categorical(L_test, num_classes=10)

# نرمال‌سازی داده‌ها (مقادیر بین 0 و 1)
X_train = (X_train - X_train.min())/(X_train.max() - X_train.min())
X_test = (X_test - X_test.min())/(X_test.max() - X_test.min())

# ساخت مدل شبکه عصبی کانولوشنی
Model = Sequential()
Model.add(Conv2D(32, (3,3), input_shape =input_shape, activation = 'relu' )) # لایه کانولوشن با 32 فیلتر
Model.add(Conv2D(64, (3,3), activation = 'relu'))                            # لایه کانولوشن با 64 فیلتر
Model.add(MaxPooling2D((2,2)))                                              # کوچک‌سازی ویژگی‌ها
Model.add(Dropout(0.25))                                                    # Dropout برای جلوگیری از overfitting
Model.add(Flatten())                                                        # تخت کردن داده‌ها
Model.add(Dense(128, activation = 'relu'))                                  # لایه کاملاً متصل با 128 نرون
Model.add(Dropout(0.5))                                                     # Dropout بیشتر
Model.add(Dense(10, activation = 'softmax'))                                # خروجی با 10 کلاس (اعداد 0 تا 9)

Model.summary()  # نمایش معماری شبکه

# کامپایل مدل با Adam و تابع خطای crossentropy
Model.compile(Adam(),
              loss = 'categorical_crossentropy',
              metrics= ['accuracy'])

# آموزش مدل روی داده‌ها
h = Model.fit(X_train, L_train, epochs= 5, batch_size = 64)

# رسم نمودار خطا
plt.plot(h.history['loss'])
plt.title('The loss of training model')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.show()

# رسم نمودار دقت
plt.plot(h.history['accuracy'])
plt.title('The accuracy of training model')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.show()

# پیش‌بینی روی داده‌های تست
Output_model = Model.predict(X_test, batch_size=64)
Output_model = np.argmax(Output_model, axis=1)  # گرفتن برچسب نهایی (کلاس بیشترین احتمال)
#Output_model = Model.predict_classes(X_test, batch_size=64)  # روش قدیمی‌تر

# محاسبه خطا و دقت روی تست
Score = Model.evaluate(X_test, L_test, batch_size= 64)
print("Test Loss: ", Score[0])
print("Test accuracy: ", Score[1])


index = 9  # شماره تصویر دلخواه در X_test
img = X_test[index].reshape(X_test.shape[1], X_test.shape[2]) * 255  # تبدیل به مقیاس 0-255
img = img.astype("uint8")  # تبدیل نوع داده برای نمایش با OpenCV
cv2.imshow("Test Image", img)
cv2.waitKey(0)   # منتظر فشردن یک کلید
cv2.destroyAllWindows()